## Everything, Everywhere, All at Once:
### A study in finding the hidden stories in chaotic datasets
**Scenario:**<br><br>
As data anlystis, we were tasked by some big movie executives to help them determine what kind of movie they should be working on next. Based on IMDb data we are looking to discover trends and relationships in the data that can help guide decisions on which project the movie studio should make next.

### Questions for which we seek answers:
1. How do movie genre trends change over time?<br>
2. Actors impacts on ratings count and average<br>
3. How popular are movies broken down by vote count?<br>
4. Do certain genres receive more votes or get better ratings?<br>
5. Do any correlations exist between movies aspects (e.g. Total Votes, Ratings, Runtime, etc.)

### Sections:
* [Exploratory Data Analysis](#eda)
* [Data Cleaning](#data-cleaning)

#### Exploratory Data Analysis <a class="anchor" id="eda"></a>

First we need to import our libraries and dataframes we will be working with from IMDb.

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
basics_df = pd.read_csv('Data/title.basics.tsv', sep='\t')
ratings_df = pd.read_csv('Data/title.ratings.tsv', sep='\t')

In [4]:
names_df = pd.read_csv('Data/name.basics.tsv', sep='\t')
principals_df = pd.read_csv('Data/title.principals.tsv', sep='\t')

We begin by looking at each data set to determine the datatypes and information available.

In [5]:
ratings_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2064
1,tt0000002,5.6,279
2,tt0000003,6.5,2038
3,tt0000004,5.4,180
4,tt0000005,6.2,2799


In [7]:
principals_df.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0005690,producer,producer,\N
3,tt0000001,4,nm0374658,cinematographer,director of photography,\N
4,tt0000002,1,nm0721526,director,\N,\N


In [8]:
names_df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"


The basics and ratings databases include a shared variable, so we can combine them

In [6]:
basics_ratings_df = basics_df.merge(ratings_df, how='inner', on='tconst')
basics_ratings_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.7,2064
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",5.6,279
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,5,"Action,Adventure,Animation",6.5,2038
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",5.4,180
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.2,2799


### Data Cleaning <a class="anchor" id="data-cleaning"></a>
Cleaning the data for easier analysis and looking for the story

First, our clients are only looking for data on movies, so we can remove all other rows.

In [9]:
basics_ratings_df.titleType.unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvEpisode', 'tvSeries',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame'], dtype=object)

In [10]:
movies_df = basics_ratings_df[basics_ratings_df.titleType == 'movie']

Then, we can determine variable types and correct as needed

In [11]:
movies_df.describe()

,averageRating,numVotes
count,314509.000000,3.145090e+05
mean,6.167837,3.621551e+03
std,1.360726,3.623180e+04
min,1.000000,5.000000e+00
25%,5.300000,1.900000e+01
50%,6.300000,6.100000e+01
75%,7.100000,3.130000e+02
max,10.000000,2.920364e+06


In [12]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 314509 entries, 8 to 1463604
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          314509 non-null  object 
 1   titleType       314509 non-null  object 
 2   primaryTitle    314509 non-null  object 
 3   originalTitle   314509 non-null  object 
 4   isAdult         314509 non-null  object 
 5   startYear       314509 non-null  object 
 6   endYear         314509 non-null  object 
 7   runtimeMinutes  314509 non-null  object 
 8   genres          314509 non-null  object 
 9   averageRating   314509 non-null  float64
 10  numVotes        314509 non-null  int64  
dtypes: float64(1), int64(1), object(9)
memory usage: 28.8+ MB


In [13]:
def year_to_decade(x: str) -> str:
    x = x[0:3] + '0'
    return x

In [14]:
def obj_to_int(x: str) -> int:
    x = int(x)
    return x

In [15]:
def genre_list(x:str) -> list:
    genre = x.split(',')
    return genre

In [16]:
movies_df = movies_df[~(movies_df.runtimeMinutes == '\\N')]
movies_df = movies_df[~(movies_df.startYear  == '\\N')]
movies_df = movies_df[~(movies_df.genres  == '\\N')]
movies_df = movies_df[~(movies_df.averageRating  == '\\N')]
movies_df = movies_df[~(movies_df.numVotes  == '\\N')]
movies_df.runtimeMinutes = movies_df.runtimeMinutes.apply(obj_to_int)
movies_df['startYearInt'] = movies_df.startYear.apply(obj_to_int)
movies_df['decade'] = movies_df.startYear.apply(year_to_decade)

In [17]:
movies_df.groupby('genres').size()

genres
Action                    3114
Action,Adult                 9
Action,Adult,Adventure       3
Action,Adult,Comedy          3
Action,Adult,Crime          10
                          ... 
Thriller,War,Western         1
Thriller,Western            11
War                        395
War,Western                 10
Western                   2397
Length: 1272, dtype: int64

In [18]:
movies_df = movies_df[~(movies_df.runtimeMinutes > 300)]
movies_df = movies_df[movies_df.numVotes > 99]

In [19]:
genres_df.genres = genres_df.genres.apply(genre_list)
genres_df

NameError: name 'genres_df' is not defined

In [34]:
genres_df = genres_df.genres.apply(pd.Series) \
    .merge(genres_df, right_index = True, left_index = True) \
    .drop(['genres'], axis = 1) \
    .melt(id_vars = ['tconst'], value_name = 'genre') \
    .drop("variable", axis = 1) \
    .dropna()

In [37]:
genres_df.sort_values('tconst')

,tconst,genre
0,tt0000009,Romance
256295,tt0000147,Sport
128148,tt0000147,News
1,tt0000147,Documentary
128149,tt0000574,Adventure
...,...,...
128144,tt9916190,Action
384438,tt9916190,Thriller
128145,tt9916270,Thriller
128146,tt9916362,Drama


In [40]:
genres_df.groupby('genre').count()

,tconst
genre,
Action,17393
Adult,815
Adventure,10204
Animation,3038
Biography,4805
Comedy,39544
Crime,15900
Documentary,11120
Drama,65964


In [43]:
genres_df = genres_df[~(genres_df.genre.isin(['Talk-Show', 'Reality-TV', 'News', 'Music', 'History', 'Game-Show']))]

In [47]:
movies_genres_df = genres_df.merge(movies_df, how='left', on='tconst')

In [48]:
movies_genres_df

,tconst,genre,primaryTitle,startYear,runtimeMinutes,averageRating,numVotes,startYearInt,decade
0,tt0000009,Romance,Miss Jerry,1894,45,5.4,212,1894,1890
1,tt0000147,Documentary,The Corbett-Fitzsimmons Fight,1897,100,5.2,520,1897,1890
2,tt0000574,Action,The Story of the Kelly Gang,1906,70,6.0,917,1906,1900
3,tt0001892,Drama,Den sorte drøm,1911,53,5.8,270,1911,1910
4,tt0001964,Drama,The Traitress,1911,48,6.0,102,1911,1910
...,...,...,...,...,...,...,...,...,...
251625,tt9900782,Drama,Kaithi,2019,145,8.4,43342,2019,2010
251626,tt9900940,Thriller,Scrapper,2021,87,4.4,1462,2021,2020
251627,tt9904802,War,Enemy Lines,2020,92,4.6,1962,2020,2020
251628,tt9907782,Mystery,The Cursed,2021,111,6.2,17743,2021,2020


In [44]:
movies_df = movies_df.drop(['endYear', 'originalTitle', 'genres', 'titleType', 'isAdult'], axis=1)

In [20]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 128147 entries, 8 to 1463578
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          128147 non-null  object 
 1   titleType       128147 non-null  object 
 2   primaryTitle    128147 non-null  object 
 3   originalTitle   128147 non-null  object 
 4   isAdult         128147 non-null  object 
 5   startYear       128147 non-null  object 
 6   endYear         128147 non-null  object 
 7   runtimeMinutes  128147 non-null  int64  
 8   genres          128147 non-null  object 
 9   averageRating   128147 non-null  float64
 10  numVotes        128147 non-null  int64  
 11  startYearInt    128147 non-null  int64  
 12  decade          128147 non-null  object 
dtypes: float64(1), int64(3), object(9)
memory usage: 13.7+ MB


The "ordering" and "characters" columns don't provide value so we drop them.

In [21]:
principals_df = principals_df.drop(columns=(['ordering', 'characters']), axis=1)

Removing deceased actors or actresses as we can't make decisions based on people who can no longer make movies.

In [22]:
names_df = names_df[(names_df.deathYear == '\\N')]

Dropping the "deathYear" column because it no longer has any valid values.

In [23]:
names_df = names_df.drop(columns=['deathYear'], axis=1)

Ensuring any null values are dropped so we are only working with known values.

In [24]:
names_df = names_df.dropna(subset=['primaryName'])

Creates one column for a main profession to make data easier to analyze.

In [25]:
def main_profession(professions):
    professions_list = professions.split(sep=',')
    return professions_list[0]

names_df['mainProfession'] = names_df['primaryProfession'].apply(main_profession)

Determining which categories exist and which are important for analysis

In [26]:
principals_df['category'].unique()

array(['self', 'director', 'producer', 'cinematographer', 'composer',
       'editor', 'actor', 'actress', 'writer', 'production_designer',
       'archive_footage', 'casting_director', 'archive_sound'],
      dtype=object)

Removing less important categories to focus on those with the most impact to help drive analysis and decision making.

In [28]:
principals_simplified_df = principals_df[(principals_df['category'].isin(['director', 'actor', 'actress', 'writer']))]

Job category doesn't provide much value so that will be dropped as well

In [29]:
principals_cleaned_df = principals_simplified_df.drop(columns=(['job']), axis=1)